# Qantitative Analyst Case Study
## Validus

#### Assumptions
Consider a multiperiod binomial asset model for an FX spot rate S with N periods. Under this particular model, we have the following assumptions:

· the initial price of the asset is 𝑆0=1.28065;

· under the risk-neutral measure, the asset price at period j is 𝑆𝑗=(1+𝑣)𝑆𝑗−1 with probability ½, and 𝑆𝑗=(1−𝑣)𝑆𝑗−1 with probability ½, with 0<𝑣<1; and

· domestic and foreign interest rates associated with borrowing/lending a currency for a single time period is 0.

1. Implement a function which, given v and the strike K of a European put option on the asset S, expiring after N periods, returns its value V.

In [ ]:
def european_put_option(S, v, K, N):
       """
        Valuation of a European put option on asset S

        Parameters:
        - S: Initial price of asset 
        - v: risk free rate
        - K: Strike price
        - N: Periods
        
        Returns: Value of option
        
        Author: Michelle
        """
        ## To calculate todays prices we use risk neutral probabilities under risk neutral measure
        u = (1+v) # up state
        d = (1-v) # down state
        p = 0.5 # probability
        
        # Calculate the asset prices at each period
        S = [S0 * u ** j * d ** (N - j) for j in range(N + 1)]

        # Calculate the option value at expiration
        V = [max(K - S[j], 0) for j in range(N + 1)] # value of put is strike - stock price

        # Calculate the option values at earlier periods
        for i in range(N - 1, -1, -1):
            for j in range(i + 1):
                V[j] = (p * V[j] + (1 - p) * V[j + 1]) / (1 + v)
                S[j] = (1 + v) * S[j]
            del V[-1]

    return V[0]

2. Implement a function which, given the strike K and value V of a European put option on the asset S, expiring after N periods, calibrates v to match this price.

In [ ]:
def calibrate_v(K, V, N, S0):
    # Define the parameters for the multiperiod binomial asset model
    p = 0.5  # risk-neutral probability
    a = 0.0001  # lower bound of v
    b = 0.9999  # upper bound of v
    tol = 0.0001  # tolerance for convergence

    # Define the function to be solved
    def f(v):
        u = 1 + v
        d = 1 - v
        S = [S0 * u ** j * d ** (N - j) for j in range(N + 1)]
        V_cal = [max(K - S[j], 0) for j in range(N + 1)]
        for i in range(N - 1, -1, -1):
            for j in range(i + 1):
                V_cal[j] = (p * V_cal[j] + (1 - p) * V_cal[j + 1]) / (1 + r)
                S[j] = (1 + v) * S[j]
            del V_cal[-1]
        return V_cal[0] - V

    # Use the bisection method to find the value of v that matches the option value
    while b - a > tol:
        c = (a + b) / 2
        if f(c) == 0:
            return c
        elif f(a) * f(c) < 0:
            b = c
        else:
            a = c

    return (a + b) / 2

3. Implement a function which, given v, returns the expectation of max0≤𝑗≤𝑁𝑆𝑗.

In [5]:
def expectation_max_fx(v, N):
    # Define the parameters for the multiperiod binomial asset model
    S0 = 1.28065  # initial price of the asset
    u = 1 + v  # up factor
    d = 1 - v  # down factor
    p = 0.5  # risk-neutral probability

    # Calculate the asset prices at each period
    S = [S0 * u ** j * d ** (N - j) for j in range(N + 1)]

    # Calculate the maximum asset value over all periods
    max_S = max(S)

    # Calculate the expectation of the maximum asset value
    exp_max_S = max_S
    
    for i in range(N):
        exp_max_S = (p * max(u ** (N - i - 1) * S[i], d ** (N - i - 1) * S[i + 1]) +
                     (1 - p) * exp_max_S) / (1 + r)

    return exp_max_S

4. Using the GBPUSD FX spot rate paths, derived from the above binomial model for 𝑁=10 (Δτ=0.5 years),𝑣=0.05, convert the GBP cashflows (provided in the cashflow data spreadsheet) into fund currency (USD) and calculate the IRR (internal rate of return) for each of the paths. Plot the distribution of the IRR values.

In [6]:
expectation_max_fx(v = 0.05, 
                  N = 10)

1.8978197201902203

Assume we would like to buy a GBPUSD European put option to hedge our FX exposure. If the USD and GBP interest rates are 0% throughout the time horizon of the fund, calculate the option’s fair market value (premium) at trade date (t = 0). See the option details below:

· Trade date: 31/03/2024

· Expiry Date: 31/03/2029

· Notional Amount: 100,000,000 GBP

· Strike 𝐾=𝑆0=1.28065


In [4]:
import numpy as np
from scipy.stats import norm
from datetime import date

# Define the option parameters
S0 = 1.28065  # spot price of GBPUSD on trade date
K = S0  # strike price
T = (date(2029, 3, 31) - date(2024, 3, 31)).days / 365  # time to expiry in years
r = 0  # risk-free interest rate
sigma = 0.15  # volatility of GBPUSD
notional = 100000000 

# Calculate the option value using the Black-Scholes model
d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
d2 = d1 - sigma * np.sqrt(T)
V = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)

# Convert the option value to GBP
premium = V * notional

print("The fair market value of the put option at time t = 0 is:", round(premium, 2), "GBP")

The fair market value of the put option at time t = 0 is: 17060929.17 GBP


6. Calculate the IRR of the hedged portfolio, including the option premium payment you calculated in (5) and the option payoff. Plot the distribution of the hedged portfolio IRR values.